<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter5/forwardselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.12


## Import modules


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula .api import ols
import numpy as np 
print(np.__version__)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


1.19.5


## Set random seed

In [3]:
np.random.seed(12)
#np.random.seed(1200)

## Original program

In [4]:
bwt = sm.datasets.get_rdataset("birthwt","MASS").data
ftv1 = (bwt['ftv']>=1).astype(int)
ptl1 = (bwt['ptl']>=1).astype(int)

remaining_features = {'lwt', 'age', 'C(ui)', 'C(smoke)',
                      'C(ht)', 'ftv1', 'ptl1'}
selected_features = []
while remaining_features: 
  PF = []  #list of (P value, feature)
  for f in remaining_features:
    temp = selected_features + [f]  #temporary list of features
    formula = 'bwt~' + '+'.join(temp) 
    fit = ols(formula,data=bwt).fit()  
    pval= fit.pvalues[-1]
    if pval < 0.05:
       PF.append((pval,f))
  if PF:  #if not empty
     PF.sort(reverse=True)
     (best_pval, best_f) = PF.pop()
     remaining_features.remove(best_f)
     print('feature {} with p-value = {:.2E}'.
            format(best_f, best_pval))
     selected_features.append(best_f)
  else:
     break
 
formula = 'bwt~lwt+age+C(race)+ smoke'
bwt_model = ols(formula, data=bwt).fit()
print(bwt_model.summary())

formula = 'bwt~age*smoke'
bwt_model = ols(formula, data=bwt).fit()
print(bwt_model.summary())
bwt_model.conf_int()

bwt['nonsmoke'] = 1 - bwt['smoke'] 
formula = 'bwt~lwt+age*nonsmoke'
bwt_model = ols(formula, data=bwt).fit()
bwt_model.conf_int()

feature C(ui) with p-value = 7.52E-05
feature C(ht) with p-value = 1.08E-02
feature lwt with p-value = 6.01E-03
feature C(smoke) with p-value = 7.27E-03
                            OLS Regression Results                            
Dep. Variable:                    bwt   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     6.373
Date:                Fri, 19 Nov 2021   Prob (F-statistic):           1.76e-05
Time:                        17:22:20   Log-Likelihood:                -1498.4
No. Observations:                 189   AIC:                             3009.
Df Residuals:                     183   BIC:                             3028.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t| 

,0,1
Intercept,1954.252680,3615.039973
lwt,0.662190,7.369973
age,-55.220626,9.257946
nonsmoke,-1751.426898,136.557576
age:nonsmoke,6.728281,86.429335


In [5]:
bwt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 85 to 84
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   low       189 non-null    int64
 1   age       189 non-null    int64
 2   lwt       189 non-null    int64
 3   race      189 non-null    int64
 4   smoke     189 non-null    int64
 5   ptl       189 non-null    int64
 6   ht        189 non-null    int64
 7   ui        189 non-null    int64
 8   ftv       189 non-null    int64
 9   bwt       189 non-null    int64
 10  nonsmoke  189 non-null    int64
dtypes: int64(11)
memory usage: 17.7 KB
